In [1]:
import pandas as pd
import tensorflow as tf
from datasets import load_dataset
import numpy as np

2023-01-14 10:30:27.612172: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/alexanderkell/anaconda3/envs/TensorFlow/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load yelp review dataset
dataset = load_dataset('yelp_polarity', split='train')
dataset

Found cached dataset yelp_polarity (/Users/alexanderkell/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/14f90415c754f47cf9087eadac25823a395fef4400c7903c5897f55cfaaa6f61)


Dataset({
    features: ['text', 'label'],
    num_rows: 560000
})

In [3]:
# print first 5 rows
type(dataset)


datasets.arrow_dataset.Dataset

In [4]:
from transformers import AutoTokenizer, TFAutoModel
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
bert = TFAutoModel.from_pretrained("distilbert-base-cased")

2023-01-14 10:30:35.319602: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some layers from the model checkpoint at distilbert-base-cased were not used when initializing TFDistilBertModel: ['vocab_projector', 'vocab_transform', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

In [5]:
SEQ_LEN = 128

In [6]:
def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["text"], truncation=True, padding="max_length", max_length=SEQ_LEN)


tokenized_dataset = dataset.map(tokenize_dataset)

 51%|█████     | 284933/560000 [03:51<03:43, 1232.56ex/s]


KeyboardInterrupt: 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

# tf_dataset = bert.prepare_tf_dataset(dataset, tokenizer=tokenizer, label_names=['label'])
tf_dataset = tokenized_dataset.to_tf_dataset(columns=["attention_mask", "input_ids"],
    label_cols=["label"], batch_size=8)

# get shape of dataset
tf_dataset.element_spec



({'input_ids': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None),
  'attention_mask': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None)},
 TensorSpec(shape=(None,), dtype=tf.int64, name=None))

In [ ]:
# print first 5 rows
tf_dataset.take(5)



<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
input_ids = tf.keras.layers.Input(shape=(SEQ_LEN,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name='attention_mask', dtype='int32')

embeddings = bert(input_ids, attention_mask=mask)[0]  # we only keep tensor 0 (last_hidden_state)

X = tf.keras.layers.GlobalMaxPool1D()(embeddings)  # reduce tensor dimensionality
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation='relu')(X)
X = tf.keras.layers.Dropout(0.1)(X)
y = tf.keras.layers.Dense(1, activation='softmax', name='outputs')(X)  # 1 labels due to two sentiment classes

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

model.layers[2].trainable = False

optimizer = tf.keras.optimizers.Adam(0.01)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

In [ ]:
model.fit(tf_dataset.take(50), epochs=2, batch_size=8)

Epoch 1/2
50/50 [==============================] - 37s 733ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/2
50/50 [==============================] - 37s 741ms/step - loss: 0.0000e+00 - accuracy: 1.0000
